# ch05/layer_naive.py

오차역전파법


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys, os
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는')
sys.path.append(os.pardir)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는/ch05')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는/common')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는/dataset')
import numpy as np

Mounted at /content/drive


단순계층 구현

In [ ]:
#곱셈계층
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


# ch05/buy_apple.py

In [ ]:
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)  #2.2 110.00000000000001 200


2.2 110.00000000000001 200


In [ ]:
#덧셈계층
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

# ch05/buy_apple_orange.py

In [ ]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)


print(price) #715 (실수)
print(dapple_num,dapple, dorange,dorange_num, dtax ) #110 2.2 3.3 165 650 (실수)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [ ]:
## 활성화 함수 계층 구현하기 ##
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

x = np.array([[1.0 , -0.5], [-2.0, 3.0]])
print(x) #[[ 1.  -0.5]  [-2.   3. ]]
mask = ( x<= 8)
print(mask) #[[ True  True]  [ True  True]]

[[ 1.  -0.5]
 [-2.   3. ]]
[[ True  True]
 [ True  True]]


In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


In [ ]:
## Affine / Softmax 계층 구현 ##

# Affine계층
X = np.random.rand(2) #입력
W = np.random.rand(2,3) #가중치
B = np.random.rand(3) #편향

X.shape
W.shape
B.shape  #(3,)

Y = np.dot(X,W) + B

(3,)

In [ ]:
# 배치용 Affine계층
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

X_dot_W
X_dot_W + B  # array([[ 1,  2,  3], [11, 12, 13]])

dY = np.array([[1,2,3], [4,5,6]])
dY
dB = np.sum(dY, axis = 0)
dB # array([5, 7, 9])

array([5, 7, 9])

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b

        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)
        return dx

In [ ]:
## Softmax-with-Loss 계층 ##
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  #손실
        self.y = None # softmax의출력
        self.t = None # 정답레이블 (one-hot vector)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx


# ch05/two_layer_net.py

In [3]:
#오차역전파법을 적용한 신경망 구현하기

import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict



class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #1번째 층 가중치
        self.params['b1'] = np.zeros(hidden_size) ##1번째 층 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss() #마지막 층

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x:입력데이터, t:정답레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:입력데이터, t:정답레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward 순전파
        self.loss(x, t)

        # backward 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

# ch05/gradient_check.py

In [ ]:
#오차역전파법으로 구한 기울기 검증

import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

#각 가중치의 차이&절대값을 구함 -> 절대값들의 평균
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.5150943604193244e-10
b1:2.173678224161356e-09
W2:4.33927435776982e-09
b2:1.400006089488648e-07


# ch05/train_neuralnet.py

In [2]:
#오차역전파법을 사용한 학습 구현하기

import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

# 데이터읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

#hyperParam
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100 #미니배ㅣ 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1) #1epoch당 반복수

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    #오차역전파법으로 기울기구함
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    #학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    #1epoch당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))


NameError: name 'TwoLayerNet' is not defined